# Lab: Model Calibration and Evaluation

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

In [ ]:
using Cropbox

### `Julia` and `Cropbox` documentations and tutorials
It takes time, practice, and frustration to become good at something and modeling is no exception. But there are pain relieavers; resources that can ease the pain to some extent. Listed below are some of the online resources I find useful for using `Julia` and `Cropbox`. In general, trying the examples provided with the documentation can be quite helpful even if they don't seem to make much sense at first.
- `Julia` language documentation: https://docs.julialang.org/en/v1/
- `Cropbox` documentation and tutorials: https://cropbox.github.io/Cropbox.jl/stable/ (while this documentation is not as complete as we like it to be, it can save you time and trouble)
- *Calculus with Julia* package: https://jverzani.github.io/CalculusWithJuliaNotes.jl/ (great resources for doing math in Julia; *Precalculus Concepts* chapter is handy)
- *DataFrames* package: https://dataframes.juliadata.org/stable/ (useful package for data handling similar to `dplyr` in R)
- Use `?` to see online doc in the notebook environment (e.g., `?plot`)

In [ ]:
new_install = false
if new_install
    import Pkg
    Pkg.add("CSV")
    Pkg.add("DataFrames")
    Pkg.add("DataFramesMeta")
    Pkg.add("StatsBase")    
    Pkg.add("Dates")
    Pkg.add("Cairo")
    Pkg.add("Fontconfig")
end

If you get errors running the cell below, some or all of the packages may not have been installed. In that case, change `new_install` to be `true` and run the cell abov to add those packages to the current kernel running in this environment.  

In [ ]:
using DataFrames
using DataFramesMeta
using StatsBase
using CSV
using Dates, Cairo, Fontconfig

## EX 1. Douglas-fir height growth ##
Let's revisit the Douglas-fir height data and estimate the parameter values (that is, calibrate or fit the model) for the exponential growth and logistic growth functions. In this exercise, we will assume that the trees began growing from 1980 as an even aged population.

- Linear growth
\begin{equation}
    \label{eq:DF_lin}
    \frac{dH}{dt} = a \\
    H =H_0 + a \cdot t 
\end{equation}

- Exponential growth
\begin{equation}
    \label{eq:DF_exp}
    \frac{dH}{dt} = rH \\
    H =H_0 \cdot e^{rt}
\end{equation}

- Logistic growth
\begin{equation}
\begin{split}
    \label{eq:DF_logis}
    \frac{dH}{dt} = rH\left(1 - \frac{H}{H_f} \right) \\
    H = \frac{H_f}{1 + (\frac{H_f}{H_0}-1)~e^{-rt}}
\end{split}
\end{equation}

1. Identify the parameters in the three growth models: *1) linear, 2) exponential*, and *3) logistic*, use either the differential equation or analytical solution of each model and calibrate the model against the observed Douglas-fir height data from the previous labs and other information needed to calibrate the model.  
2. How do the parameter estimates compare with those we estimated by intuition, visual inspection of data, or picking two data points over an interval of time in the previous labs?
3. Plot the observed and predicted values for both models. Interpret and discuss the results. 

In [ ]:
@system DF_lin(Controller) begin    
    t(context.clock.time) ~ track(u"yr")
    a:  absolute_growth_rate   ~ preserve(parameter, u"m/yr")
    H0: initial_height         ~ preserve(parameter, u"m")
    yr: year           => 1    ~ accumulate(u"yr", init = 1980u"yr")    
#   H(a): height       => a    ~ accumulate(u"m", init = H0)
    H(a): height               ~ accumulate(u"m", init = H0)
end

In [ ]:
@system DF_exp(Controller) begin
    t(context.clock.time) ~ track(u"yr")
    H0: initial_height ~ preserve(parameter, u"m")
    r: relative_growth_rate ~ preserve(parameter, u"m/m/yr")
#    H(r, t, H0) => H0*exp(r*t) ~ track(u"m")
    yr: year => 1 ~ accumulate(u"yr", init = 1980u"yr")
    H(r,H) => r*H ~ accumulate(u"m", init = H0)
end

In [ ]:
@system DF_logi(Controller) begin
    t(context.clock.time) ~ track(u"yr")
    H0: initial_height ~ preserve(parameter, u"m")
    Hf: potential_height ~ preserve(parameter, u"m")
    r: relative_growth_rate ~ preserve(parameter, u"m/m/yr")
    H(r, t, H0, Hf) => Hf/(1+(Hf/H0 - 1.0)*exp(-r*t)) ~ track(u"m")
    yr: year => 1 ~ accumulate(u"yr", init = 1980u"yr")
#    H(r,H,Hf) => r*H*(1-H/Hf) ~ accumulate(u"m", init = H0)
end

First, let's set the time step to be yearly. Since this will be common for all models we configure it first and embed it to each system we create.

In [ ]:
#c1 = @config(:Clock => :step => 1u"yr")
c1 = @config(:Clock => (;step = 1u"yr",))

Create a configuration for the linear model. Note we include `c1` to be merged with new configuration `c1a`.  

In [ ]:
c1a = @config(c1,
:DF_lin => (;    
    H0 = 1.42,
    a = 0.25,
    )
)
    

In [ ]:
c1b = @config(c1,
:DF_exp => (;    
    H0 = 1.42,
    r = 0.07,
    )
)
    

In [ ]:
c1c = @config(c1,
:DF_logi => (;
    H0 = 1.42,
    r = 0.07,        
    Hf = 70.0,
    )
)
    

In [ ]:
visualize([DF_lin, DF_exp, DF_logi], :yr, :H; configs = [c1a, c1b, c1c], stop = 30u"yr", kind = :line)

In [ ]:
DF_obs = CSV.read("../Module02-Growth_functions/PSME-height.csv", DataFrame) |> unitfy;

Let's check the data structure by looking at the first 3 rows of all variables (i.e., columns)

In [ ]:
DF_obs[1:3, :]

It looks like the Douglas-fir height data includes only two columns of data: `year` and `height`. We will add a third column for tree age because the growth models we defined are driven by `age` in years since 1983 as `time` variable, not the calendar years. We use `select` function from the `DataFrames` package. The same can also be achieved by adding a new column to the dataframe as a 2 dimensional array without using the function.

In [ ]:
?select;

In [ ]:
select!(DF_obs, :, :year => (x -> x .- 1980u"yr") => :age)
# DF_obs[:, :age] .= DF_obs[:, :year] .- 1983u"yr"
DF_obs;

## Model calibration with `calibrate()` method
Cropbox provides `calibrate()` function to estimate a set of parameter values best describing the dataset it's provided with. Internally, it relies on [BlackBoxOptim.jl](https://github.com/robertfeldt/BlackBoxOptim.jl) for global optimization methods. If you're interested in local optimization methods, refer to [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package for more information.

`calibrate()` is in a form like `calibrate(<System>, <DataFrame>; index=<..>, target=<..>, parameters=<..>, metric=<..>, optim=(; <BlackBoxOptim options>..), <simulate() options>..`. It acceps arguments similar to `simulate()` for setting up simulations. Then it also requires a data frame for ground truth data and their column names for `index` and `target` variables. Oftentimes, it is in a time series like how our dataset is recorded for corn harvest measurements. `parameters` are in a format similar to Cropbox configuration object except that each variable has a tuple of values specifying lower and upper bounds of the parameter instead of an actual value.

We first need to identify parameters, determine which parameters to calibrate and what values should be used as their ranges. We use same period of simulation (e.g., 30 yrs) as `stop` condition. The deafult metric of cost (error) function used by calibration is [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation). More advanced options controlling BlackBoxOptim can be specified with optional `optim` including maximum steps of calibration which mainly determines how long calibration process can take.

In [ ]:
fit_lin = calibrate(DF_lin, DF_obs;
    config = c1a, # comment this line and see what happens
    index = :age => :t,
    target = :height => :H,
    parameters = DF_lin => (;
        a = (0, 1.0),
        H0 = (1, 2),
    ),
    stop = 30u"yr",
 #   snap = 1u"yr",
    metric = :rmse,
    optim = (;
        MaxSteps = 1000,
        ),    
)

In [ ]:
fit_exp = calibrate(DF_exp, DF_obs;
    config = c1b, # comment this line and see what happens
    index = :age => :t,
    target = :height => :H,
    parameters = DF_exp => (;
        r = (0, 0.1),
        H0 = (1, 2),
    ),
    stop = 30u"yr",
 #   snap = 1u"yr",
    metric = :rmse,
    optim = (;
        MaxSteps = 1000,
        ),    
)

In [ ]:
fit_logi = calibrate(DF_logi, DF_obs;
    config = c1c, # comment this line and see what happens
    index = :age => :t,
    target = :height => :H,
    parameters = DF_logi => (;
        r = (0, 0.1),
        H0 = (1, 2),
     #   Hf = (60, 80),
    ),
    stop = 30u"yr",
 #   snap = 1u"yr",
    metric = :rmse,
    optim = (;
        MaxSteps = 1000,
        ),    
)

Do you think it is reasonable for us to try to calibrate $H_f$ here?

3. How do the parameter estimates compare with those we estimated by intuition, visual inspection of data, or picking two data points over an interval of time in the previous labs?
4. Plot the observed and predicted values for both models. Interpret and discuss the results. 

In [ ]:
p = visualize(DF_obs, :age, :height; xlim = (0, 30), xlab = "Age", ylab = "Height", title = "Douglas-fir Height: Linear Model")
visualize!(p, DF_lin, :t, :H; config = (c1a, fit_lin), stop = 30u"yr", kind = :line)

In [ ]:
p = visualize(DF_obs, :age, :height; xlim = (0, 30), xlab = "Age", ylab = "Height", title = "Douglas-fir Height: Exponential Model")
visualize!(p, DF_exp, :t, :H; config = (c1b, fit_exp), stop = 30u"yr", kind = :line)

In [ ]:
p = visualize(DF_obs, :age, :height; xlim = (0, 30), xlab = "Age", ylab = "Height", title = "Douglas-fir Height: Logistic Model")
visualize!(p, DF_logi, :t, :H; config = (c1c, fit_logi), stop = 30u"yr", kind = :line)

### Interpolation and extrapolation: Can we do that?

In [ ]:
p = visualize(DF_obs, :age, :height; xlim = (0, 30), xlab = "Age", ylab = "Height", title = "Douglas-fir Height Models")
visualize!(p, [DF_lin, DF_exp, DF_logi], :t, :H; configs = [(c1a, fit_lin), (c1b, fit_exp), (c1c, fit_logi)], stop = 30u"yr", kind = :line)

In [ ]:
p = visualize(DF_obs, :age, :height; xlim = (0, 100), xlab = "Age", ylab = "Height", ylim = (0, 80), title = "Douglas-fir Height Model Extrapolations")
visualize!(p, [DF_lin, DF_exp, DF_logi], :t, :H; configs = [(c1a, fit_lin), (c1b, fit_exp), (c1c, fit_logi)], stop = 100u"yr", kind = :line)

In [ ]:
using Cairo, Fontconfig
p[] |> Cropbox.Gadfly.PNG("DF-models.png")

## Model evaluation using `evaluate()` method and its metrics
Cropbox provides `evaluate()` function, which is in a similart form to `calibrate()`, for calculating a measure of error between observation input and simulation output. We can specify an arbitrary metric function in `metric` option that accepts a list of estimated values (`E`) and corresponding observed values (`O`) and returns a meausre of error.

#### Root Mean Square Error (RMSE)

- Eqn 3.1.

$$
\mathrm{RMSE} = \sqrt{\frac{\sum_{i=1}^N (\hat{y}_i - y_i)^2}{N}}
$$

RMSE can be defined by a simple function like this.

In [ ]:
rmse(E, O) = √mean((E .- O).^2)

Let's look at the goodness of fit of the linear model. A nice property of RMSE is that it inherits the original unit; m in this case. You might notice that the value of RMSE is the same as `fitness` output of the calibration. That is because we used `RMSE` as the objective function in the optimization process during calibration. See the calibration step above and compare.

Then call the `evaluate()` method built in `Cropbox` using our `rmse()` function as metric to calculate RMSE for our model predictions.

In [ ]:
evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = rmse,
)

#### Model Efficiency (EF)

- Eqn 3.2.

$$
\mathrm{EF} = 1 - \frac{\sum_i (\hat{y}_i - y_i)^2}{\sum_i (y_i - \bar{y})^2}
$$

Model efficiency (EF), or more specficially, Nash-Sutcliffe model efficiency coefficient (NSE) can be defined in a similar way.

In [ ]:
ef(E, O) = 1 - sum((E .- O).^2) / sum((O .- mean(O)).^2)

In [ ]:
evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = ef,
)

In fact, Cropbox already has an implementation of commonly used metric functions including RMSE, EF, and Willmott's $d_r$. They are specified by pre-defined symbols (*e.g.* `:rmse` for RMSE, `:ef` for EF, `:dr` for Willmott's revised index of agreement).

In [ ]:
evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = :rmse,
)

In [ ]:
evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = :ef,
)

In [ ]:
evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = :dr,
)

The maximum value of EF is 1 which indicates a perfect model. When EF is zero, the model has the same predictive power as mean of observations. A negative value of EF indicates the model is worse than the observation mean. `Cropbox` provides additional metrics of model evaluation such as Willmott's revised $d$ (`:dr`), mean absolute error (`:mae`), nornalized RMSE (`:nrmse`) among others.

Note that RMSE shares the same units (`m`) with the `target` variable.

Again, beware that what we just did above was not a proper **validation** of calibrated model since the dataset we used for calibration was used again for calculating a measure of error. In other words, we were just looking at the fitness of calibration itself. For validation, we need another dataset independent of calibration.

## Ex 2. Evaluating Douglas-fir height growth models

In this exercise, we will evaluate the performance of three models for their ability to account for the variabilities in the Douglas-fir height data used for *calibration*. Let's use three goodness of fit (GOF) metrics to compare model fit for calibration. 

1. Evaluate RMSE, EF, and $d_r$ for the three models.
1. Fill in the `gof` dataframe table below.
1. Save the table as a `CSV` file.



In [ ]:
DF_gof = DataFrame(:model => ["linear","exponential","logistic"], :RMSE => 0.0u"m" , :EF =>0.0, :d_r => 0.0) |> unitfy

In [ ]:
DF_gof[1, :RMSE] = evaluate(DF_lin, DF_obs;
    config = (c1a, fit_lin),
    index = :age => :t,
    target = :height => :H,
    stop = 30u"yr",
    metric = :rmse,
)

In [ ]:
DF_gof

In [ ]:
CSV.write("./DF_gof.csv", DF_gof, overwrite=true)

## Ex 3. Corn biomass accumulation

In this exercise, we will calibrate the logistic and Gompertz growth functions using experimental data from field corn research. We've seen the weather data from the corn experiment conducted in Beltsville, MD in 2002. Planting was done on May 15, 2002 with 8 $\mathrm{plants/m^2}$ of planting density. Plants were well-fertilized and irrigated as needed.

1. Calibrate the differential equations of the a) logistic and b) Gompertz growth functions against the corn total shoot biomass data collected over time. Think carefully about biological meanings of the parameters and what starting values you will use to calibrate them?
2. Plot predicted vs observed values of the models and compare.
3. Evaluate the performance of both models based on RMSE, EF, and $d_r$ as the goodness of fit metrics. Analyze and interpret the results. Discuss which model would be your choice of model for this purpose and why. 

### Loading data from a file

We can load CSV files into a table-like structure (`DataFrame`) using [CSV.jl](https://github.com/JuliaData/CSV.jl) package.

Cropbox provides a convenient feature with `unitfy()` function that automatically handle units for each column. For example, if the column name was set to "LA (cm^2)" in the CSV file, it would be converted to a column named "LA" and its underlying type incorprating units of "cm^2".

Note that we use piping operator (`|>`) to hand over the data frame returned by `CSV.read()` to `unitfy()` function. It's identical as if we called `unitfy(CSV.read(...))`.

In [ ]:
corn_data = CSV.read("corn_data.csv", DataFrame) |> unitfy;

There are many columns in the data frame as we can see with `names()` function.

In [ ]:
names(corn_data);

`DAS` means "days after sowing". We have dry weights (DW) in 'grams' for shoot (`shootDW`), stem (`StemDW`), leaf (`LfDW`), and ear (`EarDW`).

In [ ]:
visualize(corn_data, :day, [:shootDW, :StemDW, :LfDW, :EarDW])

Among them, we'll first use shoot dry weight (`shootDW`) that represents the above ground biomass (g) to calibrate the logistic growth model we discussed last week. Note that data have been collected weekly throughout the growing season to capture the entire growth phases.

### Logistic Equation
- Eqn.

$$
\frac{dW}{dt} = rW \cdot \left(1-\frac{W}{W_f} \right) \\
W = \frac{W_f}{1 + (\frac{W_f}{W_0} - 1) e^{-rt}}
$$

Here is the logistic growth model we're going to use for modeling biomass accumulation. It's basically the same as the system `Eq25` we used before.

Note that there are two ways of using exponential function in Julia. The first one is to use `exp()` function and the other is to use constant variable `ℯ` representing Euler's number with power operator `^` . To get the Unicode symbol `ℯ`, type `\euler` or first few letters like `\eu` and press tab key. `exp(-r*t)` would be equivalent to `ℯ^(-r*t)`.

- Table 3.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative (or specific or intrinsic) growth rate |
| $W_0$ | - | $\mathrm{g}$ | Seed mass or initial biomass |
| $W_f$ | - | $\mathrm{g}$ | Potential final mass of a plant |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter (state variable) |

In [ ]:
@system Logistic(Controller) begin
    t(context.clock.time) ~ track(u"d")
    
    r:  relative_growth_rate    ~ preserve(parameter, u"g/g/d")
    W0: initial_biomass         ~ preserve(parameter, u"g")
    Wf: potential_final_biomass ~ preserve(parameter, u"g")

    # W(r, W, Wf): biomass => begin
    #     r * W * (Wf - W) / Wf
    # end ~ accumulate(u"g", init=W0)

    W(t, r, W0, Wf): biomass => begin
        Wf / (1 + ((Wf / W0) - 1) * ℯ^(-r*t))
    end ~ track(u"g")
end

#### Calibration
In this case, we're calibrating three parameters, `r`, `W0`, and `Wf`, within a reasonable range of values. We use same period of simulation, 150 days, as `stop` condition. The deafult metric of cost (error) function used by calibration is [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation). More advanced options controlling BlackBoxOptim can be specified with optional `optim` including maximum steps of calibration which mainly determines how long calibration process can take.

Configure `calibrate` method to run *daily* instead of the default *hourly* time steps. This can reduce time for calibration substantially unless data to calibrate with are provided in hourly intervals.

In [ ]:
c = @config(
    :Clock => :step => 1u"d",
    )

In [ ]:
lc = calibrate(Logistic, corn_data;
    config = c,
    index = :DAS => :t,
    target = :shootDW => :W,
    parameters = Logistic => (;
        r = (0, 1),
        W0 = (0, 10),
        Wf = (0, 500),
    ),
    stop = 150u"d",
    metric = :rmse,
)

After calibration, we found a set of parameter estimates that can describe the observation reasonable well with fitness value of 30.79 which referrs to RMSE. In other words, our simulation with this particular set of parameters had an average error about 30 $\mathrm{g}$ compared to the original dataset.

The result of calibration is returned as a configuration object which can be directly plugged into `simluate()` or `visualize()`. Let's plot the result of simulation using the calibrated parameter set and compare it with the dataset used for calibration.

Before we used two-line code for overlaying observed vs predicted figures. Here is an one-line code doing the same using `visualize(<DataFrame>, <System>, <x>, <y>; <simulate() options>.., <plot() options>..)`.

In [ ]:
visualize(corn_data, Logistic, :DAS => :t, :shootDW => :W; config  = lc, stop = 150u"d", xlim = (0, 150))

Another form of `visualize(<DataFrame>, <System>, <y>; index=<..>, <simulate() options>.., <plot() options>..)` provides an 1:1 plot with observation against model estimation.

In [ ]:
visualize(corn_data, Logistic, :shootDW => :W; index = :DAS => :t, config = lc, stop = 150u"d")

### Gompertz Equation
- Diffrential equations
$$
\begin{align}
    \frac{dW}{dt} = rW \\
    \frac{dr}{dt} = -\alpha r
\end{align}    
$$

- Analytical solution
$$
W = W_0 \exp \left[ \frac{r_0}{\alpha} (1 - e^{-\alpha t}) \right]
$$

- Table 3.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $r_0$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Initial intrinsic growth rate before loss of efficiency |
| $W_0$ | - | $\mathrm{g}$ | Seed mass or initial biomass |
| $\alpha$ | - | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter (state variable) |

In [ ]:
@system Gompertz(Controller) begin
    t(context.clock.time) ~ track(u"d")

    r0: intrinsic_growth_rate => 0.24 ~ preserve(parameter, u"g/g/d")
    W0: initial_biomass       => 0.15 ~ preserve(parameter, u"g")
    α:  decay_rate            => 0.03 ~ preserve(parameter, u"d^-1")
    
# differential equations
    r(α, r): actual_growth_rate => -α*r ~ accumulate(init = r0, u"g/g/d")
    W(r, W): biomass            => r*W  ~ accumulate(init = W0, u"g")
    
# analytical solution
    # W(t, r0, W0, α): biomass => begin
    #     W0 * exp(r0 / α * (1 - ℯ^(-α*t)))
    # end ~ track(u"g")    
end

In [ ]:
gc = calibrate(Gompertz, corn_data;
    config = c,
    index = :DAS => :t,
    target = :shootDW => :W,
    parameters = Gompertz => (;
        r0 = (0, 0.5),
        W0 = (0, 1),
        α = (0, 0.1),
    ),
    stop = 150u"d",
    metric = :rmse,
)

In [ ]:
visualize(corn_data, Gompertz, :DAS => :t, :shootDW => :W; config = (c, gc), stop = 150u"d", xlim = (0, 150))

In [ ]:
visualize(corn_data, Gompertz, :shootDW => :W; index = :DAS => :t, config = (c, gc), stop = 150u"d")

### Model evaulation
Evaluate the performance of both models based on RMSE, EF, and $d_r$ as the goodness of fit metrics. Analyze and interpret the results. Discuss which model would be your choice of model for this purpose and why. 

In [ ]:
corn_gof = DataFrame(:model => ["Logistic","Gompertz","Chanter"], :RMSE => 0.0u"g" , :EF =>0.0, :d_r => 0.0) |> unitfy

In [ ]:
corn_gof[1, :RMSE] = evaluate(Logistic, corn_data;
    config = (c, lc),
    index = :DAS => :t,
    target = :shootDW => :W,
    stop = 150u"d",
    metric = :rmse,
)

In [ ]:
corn_gof

#### Do we always need to calibrate all parameters all at once?

Sometimes we may not need or wish to calibrate all parameters in a model at the same time. It may even be preferable to determine a parameter value based on the literature or prior knowledge instead of going through calibration process in some cases especially if those prior values are based on solid biological understandings and/or data are limited for reliable calibration. For example, it is relatively simple to determine the seed mass of corn before planting. Here, we will use seed mass ($W_0$) of 0.275g as an example. This value is based on our prior knowledge of the mass of a corn kernel.  This will help us reduce the number of prameters to calibrate. This might sacrifice the fitness (e.g., RMSE) a little but that's ok and we can determine how much fitness is lost, and whether it was worth it. There could be multiple ways to do this. Let's think about how we can achieve this in `Cropbox`.

In [ ]:
# Here we will force W0 to be 0.275g based on our prior knoledget of the seed mass of corn kernels. 
# This will help us reduce the number of prameters to calibrate. This might sacrifice the fitness (e.g., rmse) but that's ok.
# There could be multiple ways to do this. Here we will simply confine the range of W0 = (0.275, 0.275)

# lc = calibrate(Logistic, corn_data;
#     config = c,
#     index = :DAS => :t,
#     target = :shootDW => :W,
#     parameters = Logistic => (;
#         r = (0, 1),
#         W0 = (0.275, 0.275),
#         Wf = (0, 500),
#     ),
#     stop = 150u"d",
#     metric = :rmse,
# )

In [ ]:
# # You can also achieve the same by setting W0 value in a configuration, and add to the calibration result.
# lc = calibrate(Logistic, corn_data;

# # set W0 value in a configuration and remove it from parameters to calibrate     
#     config = (c, :Logistic => (; W0 = 0.275,)),

#     index = :DAS => :t,
#     target = :shootDW => :W,
#     parameters = Logistic => (;
#         r = (0, 1),
# #        W0 = (0.0, 0.5),
#         Wf = (0, 500),
#     ),
#     stop = 150u"d",
#     metric = :rmse,
# )


Combine `lc` configuration which includes newly calibrated parameter estimates (i.e., $r$ and $W_f$), `c` configuration for running on daily timestep, and $W_0$ value into one configuration `lc1`. We do this because `calibrate()` function creates a new configuration that includes the estimates for only those parameters that were calibrated.

In [ ]:
lc1 = @config (c, lc,
    Logistic => (;
        W0 = 0.275,
    ),
)

In [ ]:
visualize(corn_data, Logistic, :DAS => :t, :shootDW => :W; config  = lc1, stop = 150u"d", xlim = (0, 150))

## Ex 4. Corn biomass accumulation as a function of growing degree days or solar radiation 
Previously, We talked about radiation use efficiency and thermal unit (e.g., growing degree days) approaches. Let's modify our logistic model to be a function of GDD or solar radiation or GDD and compare the behavior and performance with the time dependent model we worked on above. 

We first need to import the weather data.

In [ ]:
corn_weather = CSV.read("./corn_weather.csv", DataFrame) |> unitfy;

Create a system for growing degree days and killing degree days for daily weather data.

In [ ]:
"Growing degree days and killing degree days calculator"
@system DegreeDays begin
    T:  temperature                              ~ preserve(parameter, u"°C")
    Tx: daily_max_temperature                    ~ preserve(parameter, u"°C")
    Tb: base_temperature                         ~ preserve(parameter, u"°C")
    To: optimal_temperature                      ~ preserve(parameter, u"°C")
    GDD_rating: maturity_growing_degree_rating   ~ preserve(parameter, u"K")
    
    GD(T, Tb, To): growing_degree => begin
        min(T, To) - Tb
    end ~ track(min = 0, u"K")

    GDD(GD): growing_degree_day => begin
        GD / 1u"d"
    end ~ track(u"K/d")

    cGDD(GDD): cumulative_growing_degree_day ~ accumulate(u"K")

    KD(Tx, To): killing_degree => begin
        Tx - To
    end ~ track(min = 0, u"K")

    KDD(KD): killing_degree_day => begin
        KD / 1u"d"
    end ~ track(u"K/d")

    cKDD(KDD): cumulative_killing_degree_day ~ accumulate(u"K")
end

set up a configuration for `DegreeDays` system.

In [ ]:
dd_config = @config (
    :Clock => (;
        step = 1u"d",
    ),
    :DegreeDays => (;
        :Tb => 10.0,
        :To => 30.0,
        :GDD_rating => 1500,
    ),
)

Create a system `Weather_daily` contain and handle imported weather daily weather data. We worked with this in the previous lab. 

In [ ]:
@system Weather_daily begin
    calendar(context)               ~      ::Calendar
    t(calendar.date)                ~ track::date

    data:                source_data     ~ provide(parameter, index = :date, init = t)
    Tavg:                avg_temperature ~ drive(from = data, by = :Tavg, u"°C")
    Tmin:                min_temperature ~ drive(from = data, by = :Tmin, u"°C")
    Tmax:                max_temperature ~ drive(from = data, by = :Tmax, u"°C")
    solrad:              solar_radiation ~ drive(from = data, by = :rad, u"MJ/m^2/d^1")
    
    T(Tavg):             temperature             => Tavg   ~ track(u"°C")
    Tx(Tmax):            daily_max_temperature   => Tmax   ~ track(u"°C")
    cRad(solrad):        sum_of_solar_radiation  => solrad ~ accumulate(u"MJ/m^2")
    
end

Combine configurations for `DegreeDays`, `Logistic`, and `Weather_daily` systems.

In [ ]:
c3 = @config (dd_config, lc,
    :Calendar => (
        :init => ZonedDateTime(2002, 5, 15, tz"America/New_York"),
        :last => ZonedDateTime(2002, 9, 30, tz"America/New_York"),
    ),
    :Clock => (;
        :step => 1u"d",
    ),
    :Weather_daily => (;
        :data => corn_weather, # provide a dataframe as source
    ),
)

Mixin `DegreeDays` and `Weather_daily` with `Controller`.

In [ ]:
@system WeatherSum(DegreeDays, Weather_daily, Controller)

Run simulation of `WeatherSum` and save results to create a dataframe containing the output bioclimatic variables that include GDD variables and cumulative radiation (`cRad`)

In [ ]:
corn_bioclim=select(simulate(WeatherSum; 
        config =(c3, dd_config), stop = "calendar.stop"), :t, :Tavg, :Tmin, :Tmax, :GDD, :cGDD, :KDD, :cKDD, :solrad, :cRad);

Merge `corn_data` and `corn_bioclim` dataframes by `:day` and `:t` as index and select variables needed later and put in a new datafram named `corn`. Rename the variable `:day` to `:date`. 

In [ ]:
corn = leftjoin(corn_data, corn_bioclim, on = :day => :t, makeunique = true)
select!(corn, :day => :date, :DAS, :shootDW, :LA, :LfNo, :Ht, :Tavg, :GDD, :cGDD, :solrad, :cRad);

Create a logistic model with cumulative radiataion (`solad`) or `GDD` as driving variable. Pay attention to the units.

In [ ]:
@system Logistic2(WeatherSum, Controller) begin
    r:  relative_growth_rate    ~ preserve(parameter, u"g/g/(MJ/m^2)")
    W0: initial_biomass         ~ preserve(parameter, u"g")
    Wf: potential_final_biomass ~ preserve(parameter, u"g")

# differential equation
    W(r, solrad, W, Wf): biomass => begin
        (r * W * (Wf - W) / Wf)*solrad
    end ~ accumulate(u"g", init=W0)    
    
# analytical solution
    # W(r, cRad, W0, Wf): biomass => begin
    #     Wf / (1 + ((Wf / W0) - 1) * ℯ^(-r*cRad))
    # end ~ track(u"g")
end

In [ ]:
lc2 = calibrate(Logistic2, corn;
    config = c3,
    index = :date => :t,
    target = :shootDW => :W,
    parameters = Logistic2 => (;
        r = (0, 0.01),
        W0 = (0, 1),
        Wf = (300, 400),
    ),
    stop = "calendar.stop",
    metric = :rmse,
)

In [ ]:
visualize(corn, Logistic2, :cRad, :shootDW => :W; config = (c3, lc2), stop = "calendar.stop")

#### Did the model performance improve?
Evaluate the performance `Logistic2` model in comparison with `Logistic` model using the GOF metrics. 

#### Will the perofmance improve further if we used `GDD` as driving variable instead of `solrad`?
Pair, disuss, and modify the model to run with `GDD` to test your hypothesis.

## Homework Problems

### Homework 1. Calibrate and evaluate Chanter function

This is an extension of the lab exercise on corn biomass accumulation. Calibrate the differential equation or analytical solution of Chanter equation with total shoot biomass (`shootDW`) in the corn growth data. This equation inherits the parameters from the Logistic and Gompertz functions with similar meanings. 

\begin{equation}
\begin{split}
    \label{eq:chanter_eqn}
    W &= \frac{W_0 B}{W_0 + (B-W_0)\exp{\left[-\frac{r}{\alpha}(1-e^{-\alpha t})\right]}}
\end{split}
\end{equation}

1. Calibrate the differential equation OR integrated form of the Chanter equation with total shoot biomass data (`shootDM`). $B$ parameter here may be interpreted as potential biomass representing the genetic ceiling that can be reached when no loss in growth efficiency occurs (i.e., $\alpha$ = 0). Provide the parameter estimates for best fit, and discuss their biological meanings. How many parameters did you end up calibrating and why?
2. Plot the result with DAS (days after sowing) on the x-axis and shoot DW on the y-axis with observed as points and predicted as line. 
3. Plot the observed (x-axis) vs the predicted (y-axis) with a 1:1 line.
4. Evaluate goodness of fit of this model, add as the third column in the table we worked on above, and compare with the logistic and Gompertz functions. Provide model evaluation stats (RMSE, MAE, EF, and Willmott's revised $d_r$) and discuss which of the three models you would consider the best model and select for for this case and why. 


### Homework 2. Modeling leaf area dynamics
Build a model for leaf area dynamics of a corn plant over time.  You may use information, skills, tools, and methods that you've learned so far in this class or other sources. In your response notebook, clearly identify the state variable(s), driving variable(s), and parameters. 

Evaluate your model's performance using RMSE, EF, and $d_r$. Compare your model's GOF metrics against the instructor's metrics. If your model outperforms his model, you can get extra points. Here's an example formula of how your score ($S$) may be determined for this homework problem.

$$
S = \frac{1}{4} \cdot \left(\frac{d_{r_1}}{d_{r_2}}  + \frac{\mathrm{EF_1}}{\mathrm{EF_2}} + \frac{\mathrm{RMSE_2}}{\mathrm{RMSE_1}} + \frac{p_2}{p_1}\right)
$$

Where the subscript $1$ represents student's metrics and $2$ is instructor's. Shown below shows the model behavior and performance metrics of instructor's model.


In [ ]:
visualize(corn, :date, :LA; xlim = (Date(2002, 6,1), Date(2002,9,30)))

In [ ]:
benchmark = CSV.read("./benchmark_gof.csv", DataFrame) |> unitfy